GRUPO 20 - A81705 - João Pedro Carvalho Henriques

In [1]:
from ortools.linear_solver import pywraplp

Inputs do problema:
- S - Sala 
- D - Dia 
- H - Hora
- P - Projeto
- C - Colaborador 
- reunioes - associa a cada projeto o numero de reuniões semanais
- lider - associa a cada projeto o seu lider
- colabs - associa a cada projeto os seus colaboradores(incluindo o lider)
- dispColabs- associa a cada colaborador as slots diponiveis

A familia de variáveis binárias $x_{s,d,h,p,c} = 1 $ sse o colaborador $c$, pertence ao projeto $p$, tem uma reunião num determinado dia $d$, na hora $h$, na sala $s$.

Condições e restrições do problema:
- 1 - Cada projeto tem um determinado número minimo de reuniões semanais.
- 2 - Cada colaborador só pode ser colocado num slot em que esteja disponivel.
- 3 - Cada colaborador só participa em reuniões de projetos em que está incluido.
- 4 - Cada sala, num determinado dia e hora, tem alocada no máximo um projecto.
- 5 - Cada colaborador de um projeto só pode estar numa sala.
- 6 - Minimo de 50% de colaboradores nas reuniões
- 7 -  O líder tem de participar em todas as reuniões do projeto.

In [2]:
#Inputs do problema
S,D,H,P,C = 3,3,3,3,12


reunioes = {0:2, 1:3, 2:4}
lider = {0:0, 1:1, 2:2}
colabs = {0:[0,3],1:[1,4,5,7],2:[2,9,8]}

dispColabs = {}
for c in range(C):
    dispColabs[c] = [(d,h) for d in range(D) for h in range(H)]
    

In [3]:
#Inicialização do Solver
horario = pywraplp.Solver.CreateSolver('SCIP')

#Matriz de alocação
x={}
for s in range(S):
    for d in range(D):
        for h in range(H):
            for p in range(P):
                for c in range(C):
                    x[s,d,h,p,c] = horario.BoolVar(f'x[%d,%d,%d,%d,%d]' % (s,h,d,p,c))
                    

1 - Cada projeto tem um determinado número minimo de reuniões semanais.
$$ \forall_{p<P} \sum_{s<S,d<D,h<H} x_{s,d,h,p,lider_p} \geq reunioes_p $$

In [4]:
#o numero de reunioes do lider do projeto é superior ou igual ao numero de reunioes do projeto
for p in range(P):
    horario.Add(sum( x[s,d,h,p,lider[p]] for s in range(S) for d in range(D) for h in range(H)) >= reunioes[p])



2 - Cada colaborador só pode ser colocado num slot em que esteja disponivel.
$$ \forall_{s<S,d<D,h<H,p<P,c<C} \ (d,h) \notin dispColabs_c \ \rightarrow \ x_{s,d,h,p,c} = 0 $$

In [5]:
for s in range(S):
    for d in range(D):
        for h in range(H):
            for p in range(P):
                for c in range(C):
                    if (d,h) not in dispColabs[c]:
                        horario.Add(x[s,d,h,p,c] == 0)

3-Cada colaborador só participa em reuniões de projetos em que está incluido.
$$ \forall_{s<S,d<D,h<H,p<P,c<C} \ c \notin colabs_p \ \rightarrow \ x_{s,d,h,p,c} = 0 $$

In [6]:
for s in range(S):
    for d in range(D):
        for h in range(H):
            for p in range(P):
                for c in range(C):
                    if c not in colabs[p]:
                        horario.Add(x[s,d,h,p,c] == 0)

4-Cada sala, num determinado dia e hora, tem alocada no máximo um projecto.
$$ \forall_{s<S,d<D,h<H,} \sum_{p<P}  x_{s,d,h,p,lider_p} \leq 1$$

In [7]:
for s in range(S):
    for d in range(D):
        for h in range(H):
            horario.Add(sum(x[s,d,h,p,lider[p]] for p in range(P)) <= 1)

5 - Cada colaborador de um projeto só pode estar numa sala.
$$ \forall_{d<D,h<H,p<P,c \in colabs_C} \sum_{s<S} x_{s,d,h,p,c} \leq 1 $$

In [8]:
for d in range(D):
    for h in range(H):
        for p in range(P):
            for c in colabs[p]:
                horario.Add(sum(x[s,d,h,p,c] for s in range(S)) <= 1)

6 - Minimo de 50% de colaboradores nas reuniões
$$ \forall_{d<D,p<P,s<S,h<H} \sum_{c \in colabs_p} x_{s,d,h,p,c} \geq  0.5 * len(colabs_p) $$

In [9]:
for d in range(D):
    for p in range(P):
        for s in range(S):
            for h in range(H):
                horario.Add(sum(x[s,d,h,p,c] for c in colabs[p]) >= 0.5*len(colabs[p])*x[s,d,h,p,lider[p]])

7 - O líder tem de participar em todas as reuniões do projeto.
$$ \forall_{s<S,h<H,p<P,c<C} (x_{s,d,h,p,c}=1) \rightarrow (x_{s,d,h,lider_p} = 1)  $$

In [10]:
#se um colaborador participa numa reuniao, obrigatoriamente o lider tambem participa

for s in range(S):
    for d in range(D):
        for h in range(H):
            for p in range(P):
                for c in colabs[p]:
                    horario.Add(x[s,d,h,p,lider[p]] >= x[s,d,h,p,c])
                   



In [11]:
#Procura da solução do problema
status = horario.Solve()
if status == pywraplp.Solver.OPTIMAL:
    print("SAT")
else:
    print("UNSAT")

SAT


In [12]:
#print
for d in range(D):
    print("Dia:" , d)
    for p in range(P):
        for h in range(H):
            for s in range(S):
                for c in colabs[p]:
                    if round(x[s,d,h,p,c].solution_value()) == 1:
                        print("Projeto:" , p, "Hora:", h, "Sala:", s, "Colaborador:", c)

Dia: 0
Projeto: 0 Hora: 0 Sala: 2 Colaborador: 0
Projeto: 0 Hora: 1 Sala: 1 Colaborador: 0
Projeto: 0 Hora: 2 Sala: 0 Colaborador: 0
Projeto: 1 Hora: 0 Sala: 0 Colaborador: 1
Projeto: 1 Hora: 0 Sala: 0 Colaborador: 4
Projeto: 1 Hora: 0 Sala: 0 Colaborador: 5
Projeto: 1 Hora: 0 Sala: 0 Colaborador: 7
Projeto: 1 Hora: 1 Sala: 2 Colaborador: 1
Projeto: 1 Hora: 1 Sala: 2 Colaborador: 4
Projeto: 1 Hora: 1 Sala: 2 Colaborador: 5
Projeto: 1 Hora: 1 Sala: 2 Colaborador: 7
Projeto: 1 Hora: 2 Sala: 2 Colaborador: 1
Projeto: 1 Hora: 2 Sala: 2 Colaborador: 4
Projeto: 1 Hora: 2 Sala: 2 Colaborador: 5
Projeto: 1 Hora: 2 Sala: 2 Colaborador: 7
Projeto: 2 Hora: 0 Sala: 1 Colaborador: 2
Projeto: 2 Hora: 0 Sala: 1 Colaborador: 9
Projeto: 2 Hora: 0 Sala: 1 Colaborador: 8
Projeto: 2 Hora: 2 Sala: 1 Colaborador: 2
Projeto: 2 Hora: 2 Sala: 1 Colaborador: 9
Projeto: 2 Hora: 2 Sala: 1 Colaborador: 8
Dia: 1
Projeto: 0 Hora: 0 Sala: 2 Colaborador: 0
Projeto: 0 Hora: 1 Sala: 0 Colaborador: 0
Projeto: 0 Hora: 2 S